# Scraper TOUT CineAtlas avec des fonctions

In [1]:
import re
import json
import requests
from ast import literal_eval

url = 'https://www.cineatlas.com/'

html_data = requests.get(url).text
movieData = re.findall(r'movieData = ({.*?}), movieDataByReleaseDate', html_data, flags=re.DOTALL)[0]
movieData = re.sub(r'\s*/\*.*?\*/\s*', '', movieData)   # remove comments
movieData = literal_eval(movieData) # in movieData you have now the information about the current movies

### Formater le JSON pour plus de lisibilité

In [2]:
jfile = json.dumps(movieData, indent=4)

In [3]:
data = json.loads(jfile)

## Scraper les dates

Le fichier JSON est défini date par date. Il faut d'abord isoler ces données pour les réutiliser avec les fonctions

In [4]:
dates = [ ]
for k in data:
    dates.append(k)

# ---------------    Fonctions   -------------------------

## Liens :

In [5]:
def scrape_liens(date):
    mLiens = []
    for i in data.get(date):
        mLiens.append("https://www.cineatlas.com/movie/"+str(i['url']))
    return mLiens

## Noms :

In [6]:
def scrape_noms(date):
    mNoms = []
    for i in data.get(date):
        mNoms.append((i['title']))
    return mNoms

## Images : 

In [7]:
def scrape_images(date):
    mImages = []
    for i in data.get(date):
        mImages.append((i['image-portrait']))
    return mImages
    

## Heures :

In [8]:
def scrape_heures(date):
    mHeures = []
    heures = []
    
    for i in data.get(date):
        h = []
        for j in i['times']:
            h.append(j['time'])
        heures.append(h)
    
    for i in heures:
        s = ""
        for j in i:
            s =s+str(j)+" "
        mHeures.append(s)     
        
    return mHeures    

## Descriptions :

In [9]:
from time import time
from time import sleep
from random import randint

In [10]:
from bs4 import BeautifulSoup
from requests import get

In [11]:
def scrape_descriptions(links):
    
    mDescs = []
    
    start_time = time()
    requests = 0
    
   
    for i in links:
        response = get(i)
    
        timer = randint(8,15)
        #pause de loop
        print(timer)
        sleep(timer)
    
        #renvoyer un warning pour les non 'status code : 200'
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
            
        #parser en BS
        page_html = BeautifulSoup(response.text,'html.parser')  
    
        events = page_html.find('div',class_='mobile-hide movie-details-container')
    
        if events is None:
            mDescs.append("NO DESCRIPTION")
            continue
    
        event = events.find('div',class_='movie-details')   
    
        
        #dans le trou du cul du code (aka 4eme <p> du <div> cible )
    
        mDescs.append(event.findAll('p',recursive=False)[3].text)
        
    print("** Done **") 
    
    return mDescs
    
     
    
    
    

# ------------------- TESTER TOUT LE BORDEL   ------------------

In [12]:
dates

['2019-08-31', '2019-09-01', '2019-08-30', '2019-09-02', '2019-09-03']

In [13]:
dates[0]

'2019-08-31'

In [14]:
super_prog =[]
for date in dates:
    
    a = scrape_liens(date)
    b = scrape_noms(date)
    c = scrape_images(date)
    d = scrape_heures(date)
    e = scrape_descriptions(a)
    
    prog = []
    for i,j,k,l,m in zip(b,d,c,a,e):
        #Ajouter le lieu
        t = (i,date,j,k,l,m,"Cine Atlas")
        prog.append(t)
        
    super_prog = super_prog + prog
    
    print(" \n \n ******* Palier "+str(date)+"******* \n \n")

print(" #############\n \n ######## BIG DONE ######## \n \n")
    
    

10
13
10
8
13
12
13
11
** Done **
 
 
 ******* Palier 2019-08-31******* 
 

10
8
13
14
13
12
11
8
** Done **
 
 
 ******* Palier 2019-09-01******* 
 

14
10
10
8
14
12
** Done **
 
 
 ******* Palier 2019-08-30******* 
 

12
10
11
14
10
14
** Done **
 
 
 ******* Palier 2019-09-02******* 
 

13
11
14
12
12
12
** Done **
 
 
 ******* Palier 2019-09-03******* 
 

 #############
 
 ######## BIG DONE ######## 
 



In [15]:
super_prog

[('TOY STORY 4',
  '2019-08-31',
  '10:30am ',
  'https://d10u9ygjms7run.cloudfront.net/dd2qd1xaf4pceqxvb41s1xpzs0/1567104205312_Poster.jpg',
  'https://www.cineatlas.com/movie/ST00000406-toy-story-4',
  'Woody a toujours privilégié la joie et le bien-être de ses jeunes propriétaires – Andy puis Bonnie – et de ses compagnons, n’hésitant pas à prendre tous les risques pour eux, aussi inconsidérés soient-ils. L’arrivée de Forky un nouveau jouet qui ne veut pas en être un dans la chambre de Bonnie met toute la petite bande en émoi. C’est le début d’une grande aventure et d’un extraordinaire voyage pour Woody et ses amis. Le cowboy va découvrir à quel point le monde peut être vaste pour un jouet…',
  'Cine Atlas'),
 ('Scary Stories to Tell in the Dark',
  '2019-08-31',
  '12:30am 2:55pm 9:35pm ',
  'https://d10u9ygjms7run.cloudfront.net/dd2qd1xaf4pceqxvb41s1xpzs0/1567104212392_Poster.jpeg',
  'https://www.cineatlas.com/movie/scary-stories-to-tell-in-the-dark',
  'Dans un manoir abandonné, 

## Convertir en objet pandas

In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(super_prog,columns=['noms','date','heure','image','lien','description','lieu'])

In [18]:
df

,noms,date,heure,image,lien,description,lieu
0,TOY STORY 4,2019-08-31,10:30am,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ST00000406-toy...,Woody a toujours privilégié la joie et le bien...,Cine Atlas
1,Scary Stories to Tell in the Dark,2019-08-31,12:30am 2:55pm 9:35pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/scary-stories-...,"Dans un manoir abandonné, un groupe de jeunes ...",Cine Atlas
2,WEDDING NIGHTMARE,2019-08-31,12:35am 3:10pm 10:35pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ready-or-not,La nuit de noces d’une jeune mariée tourne au ...,Cine Atlas
3,LE ROI LION,2019-08-31,10:30am 10:30am 1:00pm 3:30pm 7:10pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ST00000442-le-...,"Au fond de la savane africaine, tous les anima...",Cine Atlas
4,COMME DES BETES 2,2019-08-31,10:30am,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/comme-des-betes-2,"La suite du film d'animation ""Comme des bêtes""...",Cine Atlas
5,FAST &amp; FURIOUS:HOBBS AND SHAW,2019-08-31,12:30pm 5:10pm 7:50pm 11:50pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/fast--furioush...,"Depuis que Hobbs, fidèle agent de sécurité au ...",Cine Atlas
6,ONCE UPON A TIME IN HOLLYWOOD,2019-08-31,1:00pm 4:05pm 6:05pm 9:20pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ST00000454-onc...,Le film est présenté en compétition au Festiva...,Cine Atlas
7,Angel Has Fallen,2019-08-31,12:30pm 5:10pm 7:40pm 10:05pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/angel-has-fallen,"Victime d’un coup monté, Mike Banning, agent d...",Cine Atlas
8,TOY STORY 4,2019-09-01,10:30am,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/ST00000406-toy...,Woody a toujours privilégié la joie et le bien...,Cine Atlas
9,Scary Stories to Tell in the Dark,2019-09-01,12:30am 2:55pm 9:35pm,https://d10u9ygjms7run.cloudfront.net/dd2qd1xa...,https://www.cineatlas.com/movie/scary-stories-...,"Dans un manoir abandonné, un groupe de jeunes ...",Cine Atlas


### Serialization

In [19]:
import pickle

In [21]:
pickle_out = open("cinealtas.pickle","wb")
pickle.dump(prog,pickle_out)
pickle_out.close()

print("File pickled")

File pickled
